In [20]:
from os import environ
import requests

def get_coords(address):
    API_KEY = 'mapzen-WAXzQ3E'
    BASE_URL = 'https://search.mapzen.com/v1/search'
    l_boundary = -79.9709
    r_boundary = -78.8324
    t_boundary = 43.9889
    b_boundary = 43.4499
    resp = requests.get(BASE_URL, params = {'api_key': API_KEY, 'size': 1, 'text': address, 
                                           'boundary.rect.min_lon': l_boundary, 'boundary.rect.max_lon': r_boundary,
                                           'boundary.rect.min_lat': b_boundary, 'boundary.rect.max_lat': t_boundary})
    data = resp.json()
    bbox = data['bbox']
    pt = {'longitude': (bbox[0] + bbox[2]) / 2, 'latitude': (bbox[1] + bbox[3]) / 2  }
    return pt

In [28]:
print(len(k))

74224


In [27]:
import pickle
from pymongo import MongoClient

dbconn = MongoClient()
db = dbconn.parking.latlong

no_matches = []
real_failures = []
cam = pickle.load(open("collapsed_address_map.pkl", "rb" ))
cac = pickle.load(open("collapsed_address_count.pkl", "rb"))
k = list(cam.keys())
k.reverse()

print("Number of addresses: ",db.count())

for idx,address in enumerate(k):
    if db.find({'collapsed_address': address}).count() == 0 and address not in real_failures:
        try:
            coords = get_coords(address)
            if coords:
                record = {'collaped_address':address, 'latitude':coords['latitude'], 'longitude':coords['longitude'],
                          'addresses_associated':cam[address], 'num_associated':cac[address]}
                db.insert_one(record)
            else:
                print("No match for: "+address)
                no_matches.append(address)  
        except:
            #print("Failed on: "+address)
            real_failures.append(address)
            if len(real_failures)%1000 == 0:
                with open('failures.pkl', 'wb') as file:
                    pickle.dump(real_failures, file)
    if idx%1000 == 0:
        print(idx)

Number of addresses:  73794
0


KeyboardInterrupt: 

In [15]:
k.reverse()
k[0]

'100 BRUNEL CT'

In [21]:
print(get_coords('200 LEEDS STREET'))

{'longitude': -79.42532399999999, 'latitude': 43.664340499999994}


In [20]:
print(len(failures))
real_failures = []
for idx,address in enumerate(failures):
    if db.find({'collapsed_address': address}).count() == 0:
        try:
            coords = get_coords(address)
            if coords:
                record = {'collaped_address':address, 'latitude':coords['latitude'], 'longitude':coords['longitude'],
                          'addresses_associated':cam[address], 'num_associated':len(cam[address])}
                db.insert_one(record)
            else:
                print("No match for: "+address)
                no_matches.append(address)  
        except:
            print("Failed on: "+address)
            real_failures.append(address)  
    if idx%1000 == 0:
        print(idx)

460
0
Failed on: ABTT800
Failed on: 300 HILTON AVE
Failed on: 100 ALPINE AV
Failed on: 100 RUSHOLME PK CRES
Failed on: 300 CURZON ST
Failed on: 300 LA PLANTE AVE
Failed on: 800 SWANWICK AVE
Failed on: 100 ST CLAIR STREET W
Failed on: 2300 EGLINTON AVENUE EAST


In [24]:
print(len(real_failures))

96
